### 7.1.1 [1-sided bounds] Check whether they correctly implement the mathematics. Correct them if not.

The mathematics for 1-sided bounds are implemented correctly. Current implementaion of one-sided condifence intervals for both `binomial` and `hypergeometric` distribuion is using the clopper-pearson methods. For one-sided conculation, we only consider one side of the probability distribution, thus `cl = 1 - alpha`. If we are trying to find the lower CI for the given value (p or G), we need to find a $\hat p$ or $\hat G$ that has `cumulative P($\hat p$ or $\hat G$) < 1 - cl` in the cumulative distribution function of given distribution (`binom.cdf(x - 1, n, q)` or `hypergeom.cdf(x - 1, N, q, n)`). Then the CI of p or G will be [$\hat p$, 1] or [$\hat G$, N]. If we are trying to find the upper CI fot the given value (p or G), we need to find a $\hat p$ or $\hat G$ that has `cumulative P($\hat p$ or $\hat G$)  > cl` in the cumulative distribution function of given distribution (`binom.cdf(x, n, q)` or `hypergeom.cdf(x, N, q, n)`). Then the CI of p or G will be [0, $\hat p$] or [0, $\hat G$].

### 7.1.2 [1-sided bounds] Check the endpoints are found in a numerically stable and efficient manner. Provide a better method if not.

For efficiency, the implemented method allow users to set a p or G value. Or they will calculate and narrow the searcing bounds used for optimization, which reduce the searching time and improved the efficiency.
For stability, in both `binomial` and `hypergeometric` functions, `brentq` is used to get the root of the optimiazed function. `brentq` is a fast rppt-finding method compared to `fsolve` or `brenth`. But it only find a zero of the function f on the sign changing interval [a , b], and if the boundary are too small it will not able to find a exact value, given that it will always return 0.

### 7.2.1 [2-sided bounds] Check whether they correctly implement the mathematics. Correct them if not.

As far as I can tell, the mathematics are implemented correctly. Both of the current implementations of the two-sided binomial and hypergeometric confidence intervals are based on the Clopper-Pearson computation method. This is computed by dividing alpha by two and computing two separate confidence intervals based on the new alpha, and using the overlap as the confidence interval. The existing code is correct and optimizes for q (the total number of good objects in the population in `cl - hypergeom.cdf(x - 1, N, q, n)` for lower confidence interval, and `hypergeom.cdf(x, N, q, n) - (1 - cl)` for upper confidence interval. For binomial, q is optimized (population probability) in `cl - binom.cdf(x - 1, n, q)` for lower confidence intervals and `binom.cdf(x, n, q) - (1 - cl)` for upper confidence intervals. These are used to find the lower and upper bounds, and then combined to form 2-sided confidence intervals.

### 7.2.2 [2-sided bounds] Check the endpoints are found in a numerically stable and efficient manner. Provide a better method if not.

The current implementation of both the hypergeometric and binomial 2-sided bounds confidence intervals use a root finding optimization method called brentq. According to SciPy, "Brent (1973) claims convergence is guaranteed for functions computable within [a,b]." One of the limitations with Brentq is that it may occasionally return 0.0 when (a) in [a,b] is smaller than 1e^-3, making the return be outside the interval. Two similarly computationally expensive optimization techniques are brenth and ridder.

The existing implementation for both types of 2-sided confidence intervals include a while loop to ensure that f(a) and f(b) are of opposite signs before passing it into the optimization function to make sure the code doesn't break. 


### 7.2.4 Calculate (not simulate) the expected width of the 2-sided 95% confidence intervals for method="clopper-pearson" , method="sterne", and method="wang" for a range of values of 𝑛 and 𝑝 (for the binomial using Clopper-Pearson and Sterne) and for 𝑁, 𝐺, and 𝑛 (for the hypergeometric using Clopper-Pearson, Sterne, and Wang).

In [1]:
from permute.utils import binom_conf_interval, hypergeom_conf_interval
from math import comb
from scipy.stats import binom, hypergeom

In [3]:
def Expected_Width_binom(n, p, method):
    E = 0
    for x in range(n): 
        L, U = binom_conf_interval(n, x, cl=0.95, alternative="two-sided", p=p, method=method)
        E += (U - L) * comb(n, x) * (p ** x) * ((1 - p) **(n - x))
    return E

#### Binomial: for different sets of n and p (Method = Clopper-person and Sterne)

In [8]:
for n in [10, 50, 100]:
    for p in [0.01, 0.1, 0.2, 0.5, 0.9, 0.95, 0.99]:
        print(f'n: {n:.0f} p: {p:.2f}')
        Expected_c = Expected_Width_binom(n, p, 'clopper-pearson')
        print('Method: clopper-pearson; Expected width', Expected_c)
        Expected_s = Expected_Width_binom(n, p, 'sterne')
        print('Method: sterne;          Expected width', Expected_s)


n: 10 p: 0.01
Method: clopper-pearson; Expected width 0.3216924180863416
Method: sterne;          Expected width 0.30576429956745843
n: 10 p: 0.10
Method: clopper-pearson; Expected width 0.42334718180815284
Method: sterne;          Expected width 0.41079816492389964
n: 10 p: 0.20
Method: clopper-pearson; Expected width 0.504839119114195
Method: sterne;          Expected width 0.48119100047359986
n: 10 p: 0.50
Method: clopper-pearson; Expected width 0.5995771255126175
Method: sterne;          Expected width 0.5458818359374995
n: 10 p: 0.90
Method: clopper-pearson; Expected width 0.31578089150939526
Method: sterne;          Expected width 0.30933273888389967
n: 10 p: 0.95
Method: clopper-pearson; Expected width 0.1858308307425528
Method: sterne;          Expected width 0.18410486472992565
n: 10 p: 0.99
Method: clopper-pearson; Expected width 0.042693163582993535
Method: sterne;          Expected width 0.04258911573989701
n: 50 p: 0.01
Method: clopper-pearson; Expected width 0.08755088484

#### Hypergeometric: for different sets of N, G and n (Method = Clopper-person, Sterne and Wang)

In [2]:
def Expected_Width_hypergeom(N, G, n, method):
    E = 0
    for x in range(n): 
        L, U = hypergeom_conf_interval(n, x, N, cl=0.95, alternative="two-sided", G=G, method=method)
        E += (U - L) * hypergeom.pmf(x, N, G, N)
    return E

In [4]:
N = 20
for n in [10, 14]:
    for G in [4, 5, 6]:
        print(f'N: {N:.0f} n: {n:.0f} G: {G:.0f}')
        Expected_c = Expected_Width_hypergeom(N, G, n, 'clopper-pearson')
        print('Method: clopper-pearson; Expected width', Expected_c)
        Expected_s = Expected_Width_hypergeom(N, G, n, 'sterne')
        print('Method: sterne;          Expected width', Expected_s)
        Expected_w = Expected_Width_hypergeom(N, G, n, 'wang')
        print('Method: wang;            Expected width', Expected_w)

N: 20 n: 10 G: 4
Method: clopper-pearson; Expected width 9.0
Method: sterne;          Expected width 10.0
Method: wang;            Expected width 7.0
N: 20 n: 10 G: 5
Method: clopper-pearson; Expected width 8.0
Method: sterne;          Expected width 10.0
Method: wang;            Expected width 8.0
N: 20 n: 10 G: 6
Method: clopper-pearson; Expected width 8.0
Method: sterne;          Expected width 10.0
Method: wang;            Expected width 7.0
N: 20 n: 14 G: 4
Method: clopper-pearson; Expected width 5.0
Method: sterne;          Expected width 6.0
Method: wang;            Expected width 4.0
N: 20 n: 14 G: 5
Method: clopper-pearson; Expected width 5.0
Method: sterne;          Expected width 7.0
Method: wang;            Expected width 5.0
N: 20 n: 14 G: 6
Method: clopper-pearson; Expected width 5.0
Method: sterne;          Expected width 7.0
Method: wang;            Expected width 5.0


#### Discuss the differences among the three methods. You might consider how long it takes the methods to run and the expected lengths of the intervals as the parameters vary.

Clopper-Pearson ensures that the tails are of equal size. In Sterne's tails, they may be of different sizes, and create a potentially tighter confidence interval than Clopper-Pearson. Same for Wang's method, it offered an even tighter condidence interval but with even tails.

For the runnning time, when N is small (around < 20), both clopper-pearson and sterne method get the confidence interval in a short time. However for wang method, it tooks longer time to finishing calculating. But when N getting bigger (around > 200), clopper-person took the shortest time to finish followed by sterne and wang. Expecially, Wang took more than a minute to finish.


#### Which would you recommend, and why? If you would recommend one method over the other in some circumstances but not in others, explain why.

For smaller population, I would recommend method `sterne` and `wang`, because it has narrower estimation of the confidence inteval. But, if we have larger population, `clopper-person` will be preferable given that it can finish in a short time and offer a relatively correct confidence interval. But, if prefered a narrower CI, `wang` method should be the first choice.

### Style Checks

In [147]:
%%bash
pip install pep257

pycodestyle permute/utils.py 
pep257 permute/utils.py 
pycodestyle permute/tests/test_utils.py 
pep257 permute/tests/test_utils.py 